In [11]:
%load_ext autoreload
%autoreload 2

In [60]:
from ISR.models import RDN, RRDN
import pytesseract
import editdistance
from PIL import Image
import numpy as np
import cv2
import matplotlib.pyplot as plt
from time import gmtime, strftime
from pathlib import Path
import tqdm

pytesseract.pytesseract.tesseract_cmd = './tesseract/tesseract.exe'

## Params model and image paths

In [14]:
# Params model and path pre-train weights
weights = 'weights/pre-train/rrdn-C4-D3-G32-G032-T10-x4/Perceptual/rrdn-C4-D3-G32-G032-T10-x4_epoch299.hdf5'
C = 4
D = 3
G = 32
G0 = 32
T = 10
scale = 4
dpi = 300
type = 'RRDN'

# Image paths
time = strftime("%Y-%m-%d_%H_%M_%S", gmtime())
path = 'Experiments/Source/'
path_sr = './'
name_model = 'sr_{type}_{c}_{d}_{g}_{g0}_{t}_{scale}_{time}.jpg'

valid_dir = 'Samples/300dpi'
predict_valid_dir = '.'

lr = str(Path(path + '75.jpg'))
sr = str(Path(path_sr + name_model.format(type=type, c=C, d=D, g=G, g0=G0, t=T, scale=scale, time=time) + '.jpg'))
hr = str(Path(path + '300.jpg'))

## Load model and predict

In [3]:
%%time
if type is 'RDN':
    rdn = RDN(arch_params={'C': C, 'D': D, 'G': G, 'G0': G0, 'T': T, 'x': scale})
else:
    rdn = RRDN(arch_params={'C': C, 'D': D, 'G': G, 'G0': G0, 'T': T, 'x': scale})

rdn.model.load_weights(weights)

Instructions for updating:
Colocations handled automatically by placer.
Wall time: 4.67 s


In [ ]:
image_sr = rdn.predict(np.array(Image.open(lr)))
Image.fromarray(image_sr).save(sr, dpi=(dpi, dpi))

## Show crop images (Orginal, Model, Bicubic)

In [ ]:
paths = [hr, sr, lr]
label = ['Bicubic X{scale}'.format(scale=scale), 
         'Model X{scale}'.format(scale=scale), 
         'Original']
image_crops = path + 'crops.jpg'

In [ ]:
columns = 2
rows = 2
w0, h0 = 75, 250
w1, h1 = 500, 350
figsize = (10, 4)

In [ ]:
images = [np.array(Image.open(image)) for image in paths]
fig = plt.figure(figsize=figsize)

for i in range(1, len(images)+1):
    ax = fig.add_subplot(rows, columns, i)
    ax.set_title(label[i-1])
    dh = images[i-1].shape[0]//images[-1].shape[0]
    dw = images[i-1].shape[1]//images[-1].shape[1]
    plt.imshow(images[i-1][h0*dh:h1*dh, w0*dw:w1*dw])

plt.savefig(image_crops, dpi=dpi)
plt.show()

## OCR-metric based Tesseract-OCR


In [69]:
def OCR(y_true, y_pred):
    try:
        ocr_origin = pytesseract.image_to_string(y_true).split()
    except pytesseract.TesseractError:
        ocr_origin = ['']

    try:
        ocr_sr = pytesseract.image_to_string(y_pred).split()
    except pytesseract.TesseractError:
        ocr_sr = ['']

    return 1 - editdistance.eval(ocr_origin, ocr_sr) / ocr_origin.__len__()

In [70]:
%%time
OCR(Image.open(hr), Image.open(sr))

Wall time: 18.7 s


0.9091995221027479

In [51]:
def low_resolution(image, scale, method=cv2.INTER_CUBIC):
    return cv2.resize(image, 
                      dsize=(image.shape[1]//scale, image.shape[0]//scale), 
                      interpolation=method)

In [ ]:
ocr = []

for image in tqdm.tqdm(Path(valid_dir).glob('*.jpg')):
    ground_truth = np.array(Image.open(image))
    low_res = low_resolution(ground_truth, scale)
    Image.fromarray(rdn.predict(low_res)).save(Path(predict_valid_dir + '/' + image.name))
    ocr.append(OCR(ground_truth, Image.open(Path(predict_valid_dir + '/' + image.name))))
    

















0it [00:00, ?it/s]















1it [01:37, 97.05s/it]















2it [03:29, 101.58s/it]

Samples\300dpi\1 (10).jpg
Samples\300dpi\1 (11).jpg
Samples\300dpi\1 (12).jpg


In [ ]:
np.average(ocr)

